In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Polygon, LineString, Point
import numpy as np
import shapely.geometry as geom

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 19
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['axes.titlepad'] = 10
#plt.rcParams['axes.labelpad'] = -2
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['xtick.major.width'] = 1.6
plt.rcParams['ytick.major.width'] = 1.6

gdfs = gpd.read_file('1-buffer-match/600mRiverReach-straightBuffer-corrected-20station.shp')
gdfs = gdfs.rename(columns = {'stnmpy':'stationid'})

df_list = pd.read_csv('./3-process/5.q_kge_med_modified_q50_iqr_node_1.5_noqa_VersionD.csv')
station_list = df_list['stationid'].unique().tolist()

gdfs = gdfs[gdfs['stationid'].isin(station_list)]
gdfs = gdfs[~gdfs['stationid'].isin(['longmenzhen'])]
# gdfs = gdfs.to_crs("EPSG:32651")
gdfs = gdfs.sort_values(by='stationid',ascending=True).reset_index(drop=True)


df = pd.read_csv('./3-process/1.all_matched_points_VersionD.csv')
df['date'] = pd.to_datetime('2000-01-01') + pd.to_timedelta(df['time'], unit='s')
df['date'] = df['date'].dt.date

# 将date转换为datetime格式以便日期比较
df['date_dt'] = pd.to_datetime(df['date'])

# 筛选2025年4月及以前的数据
cutoff_date = pd.Timestamp('2025-04-30')
df_filtered = df[df['date_dt'] <= cutoff_date]

# 删除辅助列
df = df_filtered.drop(columns=['date_dt'])

df = df.drop_duplicates(subset = ['node_id'])
print(len(df))
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
point = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

point['name'] = (
    point.sort_values(['stationid', 'lon'])  # 按 stationid 和 lon 排序
    .groupby('stationid')                   # 分组
    .cumcount()                        # 生成递增编号
    .add(1)                            # 编号从 1 开始
    .astype(str)                       # 转换为字符串
    .radd('N')                         # 添加前缀 'N'
)


fig, axes = plt.subplots(4, 5, figsize=(20, 16), sharey=False)
# 将 axes 转换为一维数组
axes = axes.flatten()

for i in range(0, 2):
    axes[i].axis('off')  # 关闭画板

for i in range(5, 7):
    axes[i].axis('off')  # 关闭画板

for i in range(19, 20):
    axes[i].axis('off')  # 关闭画板
    
# for i in range(22, 25):
#     axes[i].axis('off')  # 关闭画板
    
# # 调整外部空白，以便留出空间给 legend
# fig.subplots_adjust(right=0.85)

for i  in range(len(gdfs)):
    # 每一行作为 DataFrame 提取
    gdf = gdfs.iloc[[i]]

    point_df = pd.merge(gdf[['stationid']],point,on='stationid',how='right')
    point_df = gpd.GeoDataFrame(point_df,  crs="EPSG:4326")
    point_df = point_df.set_geometry('geometry')
    station = gdf['stationid'].iloc[0]

    # first = gdf[['stnmpy','geometry_x']]
    # first_df = first.rename(columns={'geometry_x':'geometry'})
    # polygon = gpd.GeoDataFrame(gdf, crs="EPSG:4326")  # 假设当前投影是 WGS 84
    # polygon = polygon.set_geometry('geometry')

    
    n = -1
    m = 2
    if (i >= n+1)&(i < m+1):
        ax = axes[i+2]
    elif i >= m+1:
        ax = axes[i+4]    


    # 绘制底图（高分辨率地图）
    # gdf.plot(ax=ax,  alpha=0.3, color = 'white', edgecolor='black',linewidth = 2) 
    gdf.plot(ax=ax,  facecolor = 'none', edgecolor='orange',linewidth = 3) 
    # # 添加高分辨率的底图
    # ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Stamen.TonerLite)
    # 尝试更换底图源，使用OpenStreetMap
    ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.Esri.WorldImagery , attribution=False)
    point_df.plot(ax=ax, color='red', markersize=25, alpha=0.8)
    # 在点附近添加名称标签
    for j, row in point_df.iterrows():
        ax.text(
            row['lon'] ,  # 调整文本位置（横向偏移）
            row['lat'],  # 调整文本位置（纵向偏移）
            row['name'],        # 添加的文本
            # fontsize=10,        # 字体大小
            color='white',       # 文本颜色
            ha='left',          # 水平对齐方式
            va='bottom'         # 垂直对齐方式
        )
        
    # # 添加经纬度标注
    # # 取多边形的中心坐标
    # centroid = polygon.centroid
    # lat, lon = centroid.y, centroid.x
    
    # # 在地图上添加标注
    # ax.text(lon + 0.01, lat + 0.01, f'Lat: {lat:.4f}, Lon: {lon:.4f}', fontsize=12, ha='left', color='black')
    ax.set_axis_off()
    number = i+1

    ax.set_title(str(number)+' '+station.capitalize(), fontweight='bold', x=0.5, y=1.05)
    # 添加指南针和距离
    compass_size = 0.15  # 指南针的大小
    ax.annotate('', xy=(0.9, 0.9), xycoords='axes fraction', xytext=(0.9, 0.9 - compass_size), 
                arrowprops=dict(facecolor='white', edgecolor='white', width=2, headwidth=10, alpha=0.7))
    ax.annotate('N', xy=(0.9, 0.9), xycoords='axes fraction', fontsize=15, ha='center', color='white')
    
    # ax.annotate('', xy=(0.95 - compass_size, 0.95), xycoords='axes fraction', xytext=(0.95 + compass_size, 0.95), 
    #             arrowprops=dict(facecolor='black', edgecolor='black', width=2, headwidth=10, alpha=0.7))
    # ax.annotate('E', xy=(0.95 + compass_size, 0.95), xycoords='axes fraction', fontsize=15, ha='center', color='black')
    
    # 获取多边形的边界框
    min_lon, min_lat, max_lon, max_lat = gdf.total_bounds  # 获取边界框
    
    # 计算x和y方向的范围差
    x_range = max_lon - min_lon
    y_range = max_lat - min_lat
    
    # 确保纵横比一致，选择较大范围
    range_diff = max(x_range, y_range)  # 保证 x 和 y 范围大小一致

    # 设置新的坐标范围，使多边形位于中心
    mid_lon = (min_lon + max_lon) / 2
    mid_lat = (min_lat + max_lat) / 2
    
    # 设置x和y轴范围，以保持相同的范围差，并且多边形位于中心
    ax.set_xlim(mid_lon - range_diff / 2, mid_lon + range_diff / 2)
    ax.set_ylim(mid_lat - range_diff / 2, mid_lat + range_diff / 2)

    
    # 选择一个起始点，在边界框的中心偏下位置
    # 例如，取边界框中心并向下偏移一点
    start_lon = mid_lon - range_diff / 2+0.06*range_diff  # 中心经度
    start_lat = mid_lat- range_diff / 2+0.02*range_diff # 中心纬度偏下
    
    # 绘制一条大约500米的线段
    # 假设1度经度大约等于111km，在赤道附近
    # 使用经纬度差来模拟500米
    delta_lat = 500 / 111000  # 500米大约是0.0045度
    # 计算终点坐标
    end_lat = start_lat   # 向南偏移500米
    end_lon = start_lon+delta_lat  # 经度不变
    
    # 创建线段（从起始点到终点）
    line = geom.LineString([(start_lon, start_lat), (end_lon, end_lat)])
    # 创建 GeoDataFrame
    gdf_line = gpd.GeoDataFrame([line], columns=['geometry'], crs="EPSG:32651")
    gdf_line.plot(ax=ax, color='white',linewidth=8)  # 绘制线段
    gdf_line.plot(ax=ax, color='blue',linewidth=3)  # 绘制线段
    
    # 添加500米的图例
    ax.annotate(f' 500 m ', xy=(0.09, 0.09), xycoords='axes fraction',  ha='left', color='lightgrey') # edgecolor='blue',bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.3')
    # gdf_line.plot(ax=ax, color='blue',linewidth=3)  # 绘制线段


plt.tight_layout()
plt.savefig('Fig1b.png',dpi=300)
# 显示图形
# plt.show()

76


/tmp/ipykernel_6574/2219378887.py:188: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


ValueError: Image size of 1267375x1352113 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 2000x1600 with 20 Axes>